<a href="https://colab.research.google.com/github/HegdeSiddesh/cs6910_Assignment2/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 2: 
The goal of this assignment is threefold:        
(i) train a CNN model from scratch and learn how to tune the hyperparameters and visualise filters      
(ii) finetune a pre-trained model just as you would do in many real world applications              
(iii) use an existing pre-trained model for a cool application.


### Import required packages

In [ ]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import random
np.random.seed(137) # To ensure that the random number generated are the same for every iteration
import warnings
#warnings.filterwarnings("ignore")
!pip install --upgrade wandb
import wandb
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
import os
from keras.datasets import fashion_mnist
from keras.layers.convolutional import Conv2D
from keras.layers import Dense, Flatten, InputLayer
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation
from wandb.keras import WandbCallback
from google.colab.patches import cv2_imshow

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#Download and unzip the iNaturalist dataset
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip -q /content/nature_12K.zip

--2022-03-25 14:01:41--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.133.128, 74.125.140.128, 108.177.15.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.133.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G  59.3MB/s    in 85s     

2022-03-25 14:03:07 (43.1 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [ ]:
#GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

#Part A: Training from scratch

##Question 1 

Build a small CNN model consisting of  5 convolution layers. Each convolution layer would be followed by a ReLU activation and a max pooling layer. Here is sample code for building one such conv-relu-maxpool block in keras.

model = Sequential()     
model.add(Conv2D(16, (3, 3)input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

After 5 such conv-relu-maxpool blocks of  layers you should have one dense layer followed by the output layer containing 10 neurons (1 for each of the 10 classes).    

The input layer should be compatible with the images in the iNaturalist dataset.
The code should be flexible such that the number of filters, size of filters and activation function in each layer can be changed. You should also be able to change the number of neurons in the dense layer.

(a) What is the total number of computations done by your network? (assume m filters in each layer of size kxk and n neurons in the dense layer)

(b) What is the total number of parameters in your network? (assume m filters in each layer of size kxk and n neurons in the dense layer)

In [ ]:
wandb.init(project="Assignment_2", entity="hithesh-sidhesh", name="Question_1")

In [ ]:
#To find the shape of the image
os.chdir("/content/inaturalist_12K/train/Amphibia")
img=plt.imread("0012ec13b97dfbfb3dd5de8c3da95555.jpg")
print(img.shape)
print("The image consists of %i pixels" % (img.shape[0] * img.shape[1]))
plt.imshow(img);

In [ ]:
#Splitting training data into train and validation sets

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,validation_split=0.1)
IMG_SIZE = (128,128)

#CONFIG = wandb.config
#BATCH_SIZE = CONFIG.batch_size

train_generator = train_datagen.flow_from_directory(
        '/content/inaturalist_12K/train',
         target_size=IMG_SIZE,
        subset = 'training',
            batch_size=32,
            class_mode='categorical',
            shuffle = True,
        seed = 137)
print('TRAINING')
print('Number of samples', train_generator.samples)
print('Names of classes', train_generator.class_indices)
print('Number of classes', len(train_generator.class_indices))
print('Number of samples per class', int(train_generator.samples / len(train_generator.class_indices) ))

validation_generator = train_datagen.flow_from_directory(
        '/content/inaturalist_12K/train',
         target_size=IMG_SIZE,
            subset = 'validation',
            batch_size=32,
            class_mode='categorical',
            shuffle = True,
        seed = 137)
print('VALIDATION')
print('Number of samples', validation_generator.samples)
print('Names of classes', validation_generator.class_indices)
print('Number of classes', len(validation_generator.class_indices))
print('Number of samples per class', int(validation_generator.samples / len(validation_generator.class_indices) ))

#Test data
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        '/content/inaturalist_12K/val',
         target_size=IMG_SIZE,
            batch_size=32,
            class_mode='categorical',
            shuffle = True,
        seed = 137)


In [ ]:
def build_model(number_of_conv_layers,number_of_filters,kernel_size,image_shape,activation_conv,pool_size,neurons_dense,activation_dense,neurons_output,activation_output):
  model = Sequential()
  for i in range(0,number_of_conv_layers):
    model.add(Conv2D(number_of_filters[i], kernel_size[i] , input_shape= image_shape))
    model.add(Activation(activation_conv[i]))
    model.add(MaxPooling2D(pool_size=pool_size[i]))

  model.add(layers.Flatten())
  model.add(layers.Dense(neurons_dense[i], activation=activation_dense[i]))
  model.add(layers.Dense(neurons_output, activation=activation_output))

  model.summary()

  model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
number_of_conv_layers=5
number_of_filters=[128]*number_of_conv_layers
kernel_size=[(3,3)]*number_of_conv_layers
image_shape=(128,128,3)
activation_conv=['relu','relu','relu','relu','relu']
pool_size=[(2,2)]*number_of_conv_layers
neurons_dense_layer=[64]*number_of_conv_layers
activation_dense=['relu','relu','relu','relu','relu']
no_of_output_classes=10
activation_output='softmax'

model=build_model(number_of_conv_layers,number_of_filters,kernel_size,image_shape,activation_conv,pool_size,neurons_dense_layer,activation_dense,no_of_output_classes,activation_output)
model.fit(train_generator,
          epochs=2,
          validation_data=validation_generator
          )



In [ ]:
test_loss, test_acc = model.evaluate(test_generator , verbose=2)
print(test_acc)

63/63 - 23s - loss: 2.1244 - accuracy: 0.2140 - 23s/epoch - 373ms/step
0.21400000154972076


##Question 2

You will now train your model using the iNaturalist dataset. The zip file contains a train and a test folder. Set aside 10% of the training data for hyperparameter tuning. Make sure each class is equally represented in the validation data. Do not use the test data for hyperparameter tuning. 

Using the sweep feature in wandb find the best hyperparameter configuration. Here are some suggestions but you are free to decide which hyperparameters you want to explore

* number of filters in each layer : 32, 64, ...  
* filter organisation: same number of filters in all layer, doubling in each subsequent layer, halving in each subsequent layer, etc   
* data augmentation (easy to do in keras): Yes, No      
* dropout: 20%, 30% (btw, where will you add dropout? you should read up a bit on this)     
* batch normalisation: Yes, No      

Based on your sweep please paste the following plots which are automatically generated by wandb:     
* accuracy v/s created plot (I would like to see the number of experiments you ran to get the best configuration).          
* parallel co-ordinates plot              
* correlation summary table (to see the correlation of each hyperparameter with the loss/accuracy)             

Also write down the hyperparameters and their values that you sweeped over. Smart strategies to reduce the number of runs while still achieving a high accuracy would be appreciated. Write down any unique strategy that you tried.


In [ ]:
sweep_config = {
    'method': 'bayes',  # grid, random
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'values': [5, 10]
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'image_size': {
            'values': [256, 512]
        },
        'no_of_conv_layers': {
            'values': [5, 6]
        },
        'no_of_filters': {
            'values': [16, 32, 64, 128]
        },
        'kernel_size': {
            'values': [3, 4]
        },
        'dense_layer_size': {
            'values': [64, 128]
        },
        'no_of_dense_layers': {
            'values': [1, 2, 3]
        },
        'filter_organization': {
            'values': ['same', 'double', 'halve']
        },
        'data_augmentation': {
            'values': ['yes', 'no']
        },
        'dropout': {
            'values': [None, 0.15, 0.2, 0.25, 0.3]
        },
        'batch_normalization': {
            'values': ['yes', 'no']
        }
    }
}

In [ ]:
def define_model(activation_function_conv, activation_function_dense, num_filters, shape_of_filters_conv, shape_of_filters_pool, batch_norm_use, fc_layer, dropout):
    model = Sequential()
    model.add(Conv2D(num_filters[0], shape_of_filters_conv[0], input_shape=input_image_shape))
    if batch_norm_use:
        model.add(BatchNormalization())
    model.add(Activation(activation_function_conv[0]))
    model.add(MaxPool2D(pool_size=shape_of_filters_pool[0], strides = (2, 2)))

    for i in range(1, 5):
        model.add(Conv2D(num_filters[i], shape_of_filters_conv[i]))
        if batch_norm_use:
            model.add(BatchNormalization())
        model.add(Activation(activation_function_conv[i]))
        model.add(MaxPool2D(pool_size=shape_of_filters_pool[i], strides = (2, 2)))

    model.add(Flatten()) # The flatten layer is essential to convert the feature map into a column vector
    model.add(Dense(fc_layer, activation=activation_function_dense))
    model.add(Dropout(dropout)) # For regularization
    model.add(Dense(10, activation="softmax"))
    return model

def build_model(number_of_conv_layers,number_of_filters,kernel_size,image_shape,activation_conv,pool_size,neurons_dense,activation_dense,neurons_output,activation_output):
  model = Sequential()
  model.add(Conv2D(num_filters[0], shape_of_filters_conv[0], input_shape=input_image_shape))
    if batch_norm_use:
        model.add(BatchNormalization())
    model.add(Activation(activation_function_conv[0]))
    model.add(MaxPool2D(pool_size=shape_of_filters_pool[0], strides = (2, 2)))
  for i in range(0,number_of_conv_layers):
    model.add(Conv2D(number_of_filters[i], kernel_size[i] , input_shape= image_shape))
    model.add(Activation(activation_conv[i]))
    model.add(MaxPooling2D(pool_size=pool_size[i]))

  model.add(layers.Flatten())
  model.add(layers.Dense(neurons_dense[i], activation=activation_dense[i]))
  model.add(layers.Dense(neurons_output, activation=activation_output))

  model.summary()

  model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
def train():
  # Default values for hyper-parameters
    config_defaults = {
        "data_aug": True,
        "train_batch_size": 128,
        "batch_norm_use": True,
        "dropout": 0,
        "num_filters": [16, 32, 64, 128, 256],
        "fc_layer": 256,
        "shape_of_filters_conv": [(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)]
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

with wandb.init(project='Assignment_2',config = sweep_config, name="Question_2"):
      config = wandb.init().config

In [ ]:
for i in range (1,5):
  print(i)

1
2
3
4


In [ ]:
kernel_size = [(3,)*2]
print(kernel_size)

[(3, 3)]


In [ ]:
wandb.agent(sweep_id, train)